<a href="https://colab.research.google.com/github/cn23070/cn23070.github.io/blob/main/Module4EoMA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
from hashlib import sha256
from datetime import datetime
from cryptography.fernet import Fernet
import os
import getpass

class UserRole:
    ADMIN = "admin"
    REGULAR_USER = "regular_user"

class User:
    def __init__(self, user_id, username, password_hash, role):
        self.user_id = user_id
        self.username = username
        self.password_hash = password_hash
        self.role = role

    def set_password(self, password):
        self.password_hash = sha256(password.encode()).hexdigest()

    def check_password(self, password):
        return self.password_hash == sha256(password.encode()).hexdigest()

    def get_role(self):
        return self.role

    def get_user_id(self):
        return self.user_id

    def get_username(self):
        return self.username

    def get_password_hash(self):
        return self.password_hash

class Administrator(User):
    def create_artefact(self, artefact, db_manager):
        db_manager.store_artefact(artefact)

    def delete_artefact(self, artefact_id, db_manager):
        db_manager.delete_artefact(artefact_id)

    def view_artefact(self, artefact_id, db_manager):
        return db_manager.retrieve_artefact(artefact_id)

    def view_logs(self, db_manager):
        db_manager.view_logs()

class RegularUser(User):
    def upload_artefact(self, artefact, db_manager):
        if artefact.owner_id == self.user_id:
            db_manager.store_artefact(artefact)

    def modify_artefact(self, artefact, db_manager):
        existing_artefact = db_manager.retrieve_artefact(artefact.artefact_id)
        if existing_artefact and existing_artefact.owner_id == self.user_id:
            db_manager.update_artefact(artefact)

    def view_artefact(self, artefact_id, db_manager):
        artefact = db_manager.retrieve_artefact(artefact_id)
        if artefact and (artefact.owner_id == self.user_id or self.get_role() == UserRole.ADMIN):
            return artefact
        return None

class Artefact:
    def __init__(self, artefact_id, title, content, owner_id):
        self.artefact_id = artefact_id
        self.title = title
        self.content = content
        self.checksum = self.generate_checksum(content)
        self.encrypted_content = None
        self.timestamps = self.create_timestamp()
        self.owner_id = owner_id

    def generate_checksum(self, data):
        return sha256(data).hexdigest()

    def encrypt_content(self, encryption_manager):
        self.encrypted_content = encryption_manager.encrypt(self.content)

    def decrypt_content(self, encryption_manager):
        return encryption_manager.decrypt(self.encrypted_content)

    def create_timestamp(self):
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

class EncryptionManager:
    def __init__(self):
        self.key = Fernet.generate_key()
        self.cipher = Fernet(self.key)

    def encrypt(self, data):
        return self.cipher.encrypt(data)

    def decrypt(self, data):
        return self.cipher.decrypt(data)

class ChecksumManager:
    @staticmethod
    def generate_checksum(data):
        return sha256(data).hexdigest()

    @staticmethod
    def verify_checksum(data, checksum):
        return sha256(data).hexdigest() == checksum

class TimestampManager:
    @staticmethod
    def create_timestamp():
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    @staticmethod
    def update_timestamp():
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

class DatabaseManager:
    def __init__(self, db_name="secure_enclave.db"):
        self.db_name = db_name
        self.connect_to_db()

    def connect_to_db(self):
        self.db_connection = sqlite3.connect(self.db_name, timeout=10)
        self.db_cursor = self.db_connection.cursor()
        self.create_tables()

    def create_tables(self):
        self.db_cursor.execute('''CREATE TABLE IF NOT EXISTS Users (
                                    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
                                    username TEXT UNIQUE NOT NULL,
                                    password_hash TEXT NOT NULL,
                                    role TEXT NOT NULL)''')

        self.db_cursor.execute('''CREATE TABLE IF NOT EXISTS Artefacts (
                                    artefact_id INTEGER PRIMARY KEY AUTOINCREMENT,
                                    title TEXT NOT NULL,
                                    content BLOB NOT NULL,
                                    checksum TEXT NOT NULL,
                                    encrypted_content BLOB NOT NULL,
                                    timestamps TEXT NOT NULL,
                                    owner_id INTEGER NOT NULL,
                                    FOREIGN KEY(owner_id) REFERENCES Users(user_id))''')

        self.db_cursor.execute('''CREATE TABLE IF NOT EXISTS Logs (
                                    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
                                    action TEXT NOT NULL,
                                    user_id INTEGER,
                                    artefact_id INTEGER,
                                    timestamp TEXT NOT NULL,
                                    FOREIGN KEY(user_id) REFERENCES Users(user_id),
                                    FOREIGN KEY(artefact_id) REFERENCES Artefacts(artefact_id))''')

        self.db_connection.commit()

    def store_user(self, user):
        self.db_cursor.execute('''INSERT INTO Users (username, password_hash, role)
                                  VALUES (?, ?, ?)''',
                               (user.username, user.password_hash, user.role))
        self.db_connection.commit()

    def store_artefact(self, artefact):
        self.db_cursor.execute('''INSERT INTO Artefacts (title, content, checksum, encrypted_content, timestamps, owner_id)
                                  VALUES (?, ?, ?, ?, ?, ?)''',
                               (artefact.title, artefact.content, artefact.checksum,
                                artefact.encrypted_content, artefact.timestamps, artefact.owner_id))
        self.db_connection.commit()

    def retrieve_artefact(self, artefact_id):
        self.db_cursor.execute('SELECT * FROM Artefacts WHERE artefact_id = ?', (artefact_id,))
        row = self.db_cursor.fetchone()
        if row:
            return Artefact(row[0], row[1], row[2], row[6])
        return None

    def update_artefact(self, artefact):
        self.db_cursor.execute('''UPDATE Artefacts
                                  SET title = ?, content = ?, checksum = ?, encrypted_content = ?, timestamps = ?
                                  WHERE artefact_id = ?''',
                               (artefact.title, artefact.content, artefact.checksum,
                                artefact.encrypted_content, artefact.timestamps, artefact.artefact_id))
        self.db_connection.commit()

    def delete_artefact(self, artefact_id):
        self.db_cursor.execute('DELETE FROM Artefacts WHERE artefact_id = ?', (artefact_id,))
        self.db_connection.commit()

    def get_user_by_username(self, username):
        self.db_cursor.execute('SELECT * FROM Users WHERE username = ?', (username,))
        row = self.db_cursor.fetchone()
        if row:
            return User(row[0], row[1], row[2], row[3])
        return None

    def view_logs(self):
        self.db_cursor.execute('SELECT * FROM Logs')
        logs = self.db_cursor.fetchall()
        for log in logs:
            print(log)

    def close(self):
        if self.db_connection:
            self.db_connection.close()

class Logger:
    def __init__(self, db_manager):
        self.db_manager = db_manager

    def log_event(self, event, user_id=None, artefact_id=None):
        timestamp = TimestampManager.create_timestamp()
        self.db_manager.db_cursor.execute('''INSERT INTO Logs (action, user_id, artefact_id, timestamp)
                                            VALUES (?, ?, ?, ?)''',
                                         (event, user_id, artefact_id, timestamp))
        self.db_manager.db_connection.commit()

    def view_logs(self):
        self.db_manager.view_logs()

def login_user(db_manager):
    username = input("Enter username: ")
    password = getpass.getpass("Enter password: ")
    user = db_manager.get_user_by_username(username)

    if user and user.check_password(password):
        return user
    return None

def main():
    db_manager = DatabaseManager()
    logger = Logger(db_manager)

    print("Welcome to the Secure Enclave Application!")
    print("Please log in to continue.")

    admin_username = 'admin'
    admin_password = getpass.getpass("Enter admin password: ")
    regular_username = 'regular'
    regular_password = getpass.getpass("Enter regular user password: ")

    admin = Administrator(None, admin_username, '', UserRole.ADMIN)
    admin.set_password(admin_password)

    try:
        db_manager.store_user(admin)
    except sqlite3.IntegrityError:
        print("Admin username already exists. Choose a different username.")
        db_manager.close()
        return

    regular_user = RegularUser(None, regular_username, '', UserRole.REGULAR_USER)
    regular_user.set_password(regular_password)

    try:
        db_manager.store_user(regular_user)
    except sqlite3.IntegrityError:
        print("Regular user username already exists. Choose a different username.")
        db_manager.close()
        return

    while True:
        user = login_user(db_manager)
        if not user:
            print("Login failed.")
            continue

        while True:
            print(f"Logged in as {user.username} with role {user.get_role()}")
            if user.get_role() == UserRole.ADMIN:
                action = input("Choose action: [upload, delete, view_logs, logout, exit]: ")
                if action == 'upload':
                    file_path = input("Enter full path to file to upload: ")
                    if os.path.exists(file_path):
                        try:
                            with open(file_path, 'rb') as file:
                                content = file.read()
                                artefact = Artefact(None, os.path.basename(file_path), content, user.get_user_id())
                                encryption_manager = EncryptionManager()
                                artefact.encrypt_content(encryption_manager)
                                db_manager.store_artefact(artefact)
                                logger.log_event("Uploaded artefact", user_id=user.get_user_id(), artefact_id=artefact.artefact_id)
                                print(f"Uploaded artefact {artefact.title}")

                                print("Artifact Details:")
                                print(f"ID: {artefact.artefact_id}")
                                print(f"Title: {artefact.title}")
                                print(f"Checksum: {artefact.checksum}")
                                print(f"Timestamp: {artefact.timestamps}")
                        except Exception as e:
                            print(f"Error reading file: {e}")
                    else:
                        print("File does not exist.")

                elif action == 'delete':
                    try:
                        artefact_id = int(input("Enter artefact ID to delete: "))
                        db_manager.delete_artefact(artefact_id)
                        logger.log_event("Deleted artefact", user_id=user.get_user_id(), artefact_id=artefact_id)
                        print("Artefact deleted.")
                    except ValueError:
                        print("Invalid artefact ID. Please enter a numeric value.")

                elif action == 'view_logs':
                    logger.view_logs()

                elif action == 'logout':
                    print("Logged out.")
                    break

                elif action == 'exit':
                    print("Exiting application.")
                    db_manager.close()
                    return

                else:
                    print("Invalid action.")

            elif user.get_role() == UserRole.REGULAR_USER:
                action = input("Choose action: [upload, modify, view, logout, exit]: ")
                if action == 'upload':
                    file_path = input("Enter full path to file to upload: ")
                    if os.path.exists(file_path):
                        try:
                            with open(file_path, 'rb') as file:
                                content = file.read()
                                artefact = Artefact(None, os.path.basename(file_path), content, user.get_user_id())
                                artefact.encrypt_content(EncryptionManager())
                                db_manager.store_artefact(artefact)
                                logger.log_event("Uploaded artefact", user_id=user.get_user_id(), artefact_id=artefact.artefact_id)
                                print(f"Uploaded artefact {artefact.title}")

                                print("Artifact Details:")
                                print(f"ID: {artefact.artefact_id}")
                                print(f"Title: {artefact.title}")
                                print(f"Checksum: {artefact.checksum}")
                                print(f"Timestamp: {artefact.timestamps}")
                        except Exception as e:
                            print(f"Error reading file: {e}")
                    else:
                        print("File does not exist.")

                elif action == 'modify':
                    try:
                        artefact_id = int(input("Enter artefact ID to modify: "))
                        existing_artefact = db_manager.retrieve_artefact(artefact_id)
                        if existing_artefact and existing_artefact.owner_id == user.get_user_id():
                            file_path = input("Enter full path to file to modify: ")
                            if os.path.exists(file_path):
                                try:
                                    with open(file_path, 'rb') as file:
                                        content = file.read()
                                        artefact = Artefact(artefact_id, os.path.basename(file_path), content, user.get_user_id())
                                        artefact.encrypt_content(EncryptionManager())
                                        db_manager.update_artefact(artefact)
                                        logger.log_event("Modified artefact", user_id=user.get_user_id(), artefact_id=artefact_id)
                                        print(f"Modified artefact {artefact.title}")

                                        print("Updated Artifact Details:")
                                        print(f"ID: {artefact.artefact_id}")
                                        print(f"Title: {artefact.title}")
                                        print(f"Checksum: {artefact.checksum}")
                                        print(f"Timestamp: {artefact.timestamps}")
                                except Exception as e:
                                    print(f"Error reading file: {e}")
                            else:
                                print("File does not exist.")
                        else:
                            print("Artefact does not exist or you do not own this artefact.")
                    except ValueError:
                        print("Invalid artefact ID. Please enter a numeric value.")

                elif action == 'view':
                    try:
                        artefact_id = int(input("Enter artefact ID to view: "))
                        artefact = db_manager.retrieve_artefact(artefact_id)
                        if artefact:
                            print(f"Artefact: {artefact.title}, Checksum: {artefact.checksum}, Timestamp: {artefact.timestamps}")
                        else:
                            print("Artefact not found.")
                    except ValueError:
                        print("Invalid artefact ID. Please enter a numeric value.")

                elif action == 'logout':
                    print("Logged out.")
                    break

                elif action == 'exit':
                    print("Exiting application.")
                    db_manager.close()
                    return

                else:
                    print("Invalid action.")

if __name__ == "__main__":
    main()
